# 初始化

In [ ]:
import os
import time
import threading
from http.server import HTTPServer, SimpleHTTPRequestHandler
from local_packages import *
from dotenv import load_dotenv
from queue import Queue
import concurrent.futures
import random
import json
import openai
from concurrent.futures import ThreadPoolExecutor, as_completed

# 加载环境变量
dotenv_path = os.path.join(os.getcwd(), '.env')
model_path = r'D:\Joining\Models\Text2Vec_base_zh'
# 设置项目根目录和图片目录
project_root = os.path.dirname(dotenv_path)

agentopener=AgentOpener(service_type='qwen')
service=agentopener.service
js=JSProcessor()

# 分割Lean

In [ ]:
import os
import glob
import json
from pygments import lex
from pygments.lexers import LeanLexer
from pygments.token import Token

# 指定目录路径
directory_path = 'D:\\Joining\\Mathlib'

# 遍历目录及其子目录中的所有 .txt 文件
txt_files = []
for root, dirs, files in os.walk(directory_path):
    for file in files:
        if file.endswith('.txt'):
            txt_files.append(os.path.join(root, file))

# 初始化 JSON 列表
json_dict_list = []

# 遍历所有找到的 .txt 文件并进行处理
for txt_file in txt_files:
    with open(txt_file, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # 使用 LeanLexer 解析代码
    lexer = LeanLexer()
    tokens = list(lex(content, lexer))
    
    # 初始化变量
    current_segment = []
    segments = []
    start_new_segment = False
    theorem_line = ""
    para_index = 0
    
    # 遍历词法单元
    for token_type, token_value in tokens:
        if token_type == Token.Keyword.Declaration and token_value == 'theorem':
            print(1)
            if current_segment:
                segments.append(''.join(current_segment))
                para_index += 1
                json_dict_list.append({
                    "txt_file_name": os.path.relpath(txt_file, directory_path),
                    "para_content": ''.join(current_segment),
                    "para_index": para_index
                })
            current_segment = []
            start_new_segment = True
            theorem_line = token_value
        
        if start_new_segment:
            current_segment.append(token_value)
            start_new_segment = False
        elif current_segment:
            current_segment.append(token_value)
    
    # 添加最后一个段落
    if current_segment:
        segments.append(''.join(current_segment))
        para_index += 1
        para_content = ''.join(current_segment)
        theorem_line = para_content.split('\n')[0].strip()
        json_dict_list.append({
            "txt_file_name": os.path.basename(txt_file),
            "para_content": para_content,
            "para_index": para_index
        })

# 将结果写入 JSON 文件
with open('output.json', 'w', encoding='utf-8') as json_file:
    json.dump(json_dict_list, json_file, ensure_ascii=False, indent=4)